API ADMINSAT

In [6]:
import json
import requests
import geojson
import os

# URL para solicitar el token de autenticación
url_autenticacion = "https://api.adminsat.com/v2/api/applications/autenticacion/token/"
# URL para solicitar la ubicación de los activos
url_ubicacion = "https://api.adminsat.com/v2/api/applications/public/ubicacion/"

# Credenciales de usuario (proporcionadas por el cliente)
usuario = "manuel.velandia"
contrasena = "YhnTgb45"
client_id = "2FPHX5uTah9b5afo3a2Fk14aNaoGTKPIUT5GXTBk"
client_secret = "wIFBOahewJLYK9btPbiC4aDwr57oY6R9yIGXKPtf9ljeiZCXjWX4MrDjOhjbHRVYUVX9X8h3aLKVCeg4xngyWGaBcYTKgFUf9Aj0bH7Vz16dErE0lQYn2eLR1cKmTOMj"

# Cuerpo de la petición para obtener el token de autenticación
cuerpo_peticion_autenticacion = {
    "username": usuario,
    "password": contrasena,
    "client_id": client_id,
    "client_secret": client_secret,
    "grant_type": "password"
}

# Cabecera de la petición para obtener el token de autenticación
cabecera_autenticacion = {
    "Content-Type": "application/json"
}

# Hacer una solicitud POST para obtener el token de autenticación
respuesta_autenticacion = requests.post(url_autenticacion, json=cuerpo_peticion_autenticacion, headers=cabecera_autenticacion)

# Verificar si la solicitud fue exitosa (código de estado 200)
if respuesta_autenticacion.status_code == 200:
    # Extraer el token de autenticación de la respuesta JSON
    access_token = respuesta_autenticacion.json()["access_token"]
    # Cabecera de la petición para obtener la ubicación de los activos
    cabecera_ubicacion = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    # Hacer una solicitud GET para obtener la ubicación de los activos
    respuesta_ubicacion = requests.get(url_ubicacion, headers=cabecera_ubicacion)
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if respuesta_ubicacion.status_code == 200:
        # Extraer la información de ubicación de la respuesta JSON
        ubicaciones = respuesta_ubicacion.json()
        
        # Crear una lista para almacenar las características GeoJSON
        features = []
        
        # Leer el archivo existente si existe
        archivo_geojson = "ubicaciones.geojson"
        if os.path.exists(archivo_geojson):
            with open(archivo_geojson, "r") as archivo:
                feature_collection_existente = geojson.load(archivo)
                features.extend(feature_collection_existente["features"])
        
        # Iterar sobre cada ubicación y crear una característica GeoJSON para cada una
        for ubicacion in ubicaciones:
            # Crear un diccionario de propiedades para la ubicación
            propiedades = {
                "Nombre": ubicacion["nombre"],
                "Placa": ubicacion["placa"],
                "Velocidad": ubicacion["velocidad"],
                "Calidad GPS": "Buena" if ubicacion["calidad_gps"] else "Mala",
                "Ignición": "Encendido" if ubicacion["ignicion"] else "Apagado" if ubicacion["ignicion"] is not None else "Desconocido",
                "Porcentaje de batería": ubicacion["bateria"],
                "Altitud": ubicacion["altitud"],
                "Fecha y hora del equipo": ubicacion["fecha_hora_equipo"],
                "Fecha y hora del servidor": ubicacion["fecha_hora_servidor"],
                "Evento": ubicacion["evento_nombre"],
                "Identificador": ubicacion["identificador"],
                "Odómetro": ubicacion["odometro"]
            }
            
            # Crear una geometría Point usando las coordenadas de latitud y longitud
            geometria = geojson.Point((ubicacion["longitud"], ubicacion["latitud"]))
            
            # Crear una característica GeoJSON con las propiedades y la geometría
            feature = geojson.Feature(geometry=geometria, properties=propiedades)
            
            # Agregar la característica a la lista de características
            features.append(feature)
        
        # Crear una colección FeatureCollection con todas las características
        feature_collection = geojson.FeatureCollection(features)
        
        # Escribir la colección FeatureCollection en el archivo .geojson
        with open(archivo_geojson, "w") as archivo:
            geojson.dump(feature_collection, archivo, indent=4)
            
        print("Archivo .geojson creado correctamente.")
            
    else:
        print("Error al obtener la ubicación de los activos:", respuesta_ubicacion.status_code)
else:
    print("Error al obtener el token de autenticación:", respuesta_autenticacion.status_code)


Archivo .geojson creado correctamente.


In [3]:
import requests
import json

# Parámetros de autenticación de ArcGIS
username = 'gis.colombia_mv'
password = 'Marviva2022#1'

# Obtener token de autenticación de ArcGIS
def get_token(username, password):
    token_url = 'https://www.arcgis.com/sharing/rest/generateToken'
    params = {
        'f': 'json',
        'username': username,
        'password': password,
        'referer': 'https://www.arcgis.com'
    }
    response = requests.post(token_url, data=params)
    response_json = response.json()
    if 'token' in response_json:
        return response_json['token']
    else:
        print("Error al obtener el token:", response_json)
        exit()

token = get_token(username, password)
print(f"Token obtenido: {token}")

# ID del ítem existente
item_id = '71a9955bd81c410186b6914c408dafb9'

# URL del servicio de ArcGIS Online donde actualizarás los datos
update_url = f'https://www.arcgis.com/sharing/rest/content/users/{username}/items/{item_id}/update'

# Actualizar archivo GeoJSON en ArcGIS Online
files = {'file': open('ubicaciones.geojson', 'rb')}  # Reemplaza con la ruta a tu archivo GeoJSON local
params = {
    'f': 'json',
    'token': token
}
response = requests.post(update_url, files=files, data=params)
response_data = response.json()

# Verifica si la actualización fue exitosa
if 'success' in response_data and response_data['success']:
    print(f"Archivo actualizado exitosamente. ID del ítem: {item_id}")
else:
    print("Error al actualizar el archivo:", response_data)
    exit()


Token obtenido: mzFcMRqhxzPAoRJavp2MJqEXElSWk0EtX6VxTnbJ9Y4ifjBXZXyChSgD-nchnr81DPpsODb8tDZHb_J69t9hNEgwwMQtVMDJafFrtT8WCzVeXs9EKGm21cj-cTxIQPMD3wRkQ8xiwXuzHyBGpiPSspe3RN_vEc2Z_XlHXcbpAA1YjiwELAapTfnQFjASHpbz
Archivo actualizado exitosamente. ID del ítem: 71a9955bd81c410186b6914c408dafb9
